In [26]:
!pip install requests beautifulsoup4

In [27]:
import requests
from bs4 import BeautifulSoup

with open('/content/url&crawled_data/urls.txt', 'r') as f:
    links = [line.strip() for line in f.readlines() if line.strip()]  # Skip empty lines

crawled_data = []

for link in links:
    try:
        response = requests.get(link)
        response.raise_for_status() 

        soup = BeautifulSoup(response.text, 'html.parser')

        text_data = soup.get_text()

        crawled_data.append(text_data)
    except requests.exceptions.RequestException as e:
        print(f"Error crawling {link}: {e}")

with open('/content/url&crawled_data/crawled_data.txt', 'w', encoding='utf-8') as f:
    for data in crawled_data:
        f.write(data + '\n\n')

In [28]:
import re

with open('/content/url&crawled_data/crawled_data.txt', 'r', encoding='utf-8') as f:
    data = f.read()

data = re.sub(r'<.*?>', '', data)

data = re.sub(r'\n\s*\n', '\n', data)

data = re.sub(r'[^a-zA-Z0-9\u0900-\u097F\s]', '', data)

data = re.sub(r'[a-zA-Z]', '', data)

data = re.sub(r'[0-9]', '', data)

with open('/content/url&crawled_data/preprocessed_data.txt', 'w', encoding='utf-8') as f:
    f.write(data)


In [29]:
with open('/content/url&crawled_data/preprocessed_data.txt', 'r', encoding='utf-8') as f:
    lines = f.readlines()

with open('preprocessed_data.txt', 'w', encoding='utf-8') as f:
    for line in lines:
        if line.strip():  # Check if the line is not empty
            f.write(line)

In [30]:
import os

# Define the input file and output directory
input_file = '/content/url&crawled_data/preprocessed_data.txt'
output_dir = '/content/pre_processed_corpus'

if not os.path.exists(output_dir):
    os.makedirs(output_dir)

with open(input_file, 'r', encoding='utf-8') as f:
    data = f.read()

sentences = [sentence.strip() for sentence in data.split('\n') if sentence.strip()]  # Remove empty lines

file_counter = 1
sentence_counter = 0

file_object = open(os.path.join(output_dir, f'corpus_{file_counter}.txt'), 'w', encoding='utf-8')

# Write each sentence to a file
for sentence in sentences:
    file_object.write(sentence + '\n')
    sentence_counter += 1
    if sentence_counter % 2000 == 0:  
        file_object.close()
        file_counter += 1
        file_object = open(os.path.join(output_dir, f'corpus_{file_counter}.txt'), 'w', encoding='utf-8')

# Close the last file object
file_object.close()

# BAG OF WORDS

## FROM SCRATCH

In [31]:
import pprint

In [38]:
import os
import string
from collections import Counter
import pandas as pd
import pprint
import csv

# Define the directory path
dir_path = '/content/pre_processed_corpus'

preprocessed_documents = []

for filename in os.listdir(dir_path):
    if filename.startswith('corpus_') and filename.endswith('.txt'):
        file_path = os.path.join(dir_path, filename)
        with open(file_path, 'r', encoding='utf-8') as f:
            document = f.read()
            # Convert to lower case
            document = document.lower()
            # Remove punctuation
            document = ''.join(c for c in document if c not in string.punctuation)
            # Split into words
            words = document.split(' ')
            preprocessed_documents.append(words)

frequency_list = []

for document in preprocessed_documents:
    frequency_list.append(Counter(document))

for i, frequency in enumerate(frequency_list):
    print(f'Frequency list for corpus_{i+1}.txt:')
    pprint.pprint(frequency)
    print()

df = pd.DataFrame()

for i, frequency in enumerate(frequency_list):
    df[f'corpus_{i+1}'] = pd.Series(frequency)

df = df.T

# Save the DataFrame to a CSV file
df.to_csv('/content/output/Bag_of_words_scratch.csv', index=True, encoding='utf-8')

print(df)

with open('/content/output/vocabulary_scratch.txt', 'w', encoding='utf-8') as f:
    for col in df.columns:
        f.write(col + '\n')

Streaming output truncated to the last 5000 lines.
         'वादेदार\nपुरुषोत्तम': 3,
         'टंडन\nरामस्वामी': 3,
         'वेंकटरमण\nराहुल': 3,
         'सांकृत्यायन\nराजेन्द्र': 3,
         'प्रसाद\nराम': 3,
         'बिस्मिल\nरानी': 3,
         'लक्ष्मीबाई\nरासबिहारी': 3,
         'बोस\nसहजानंद': 3,
         'सरस्वती\nसंगोली': 3,
         'रायण्णा\nसरोजिनी': 3,
         'नायडू\nसत्यपाल': 3,
         'डांग\nशुजाउद्दौला\nश्यामजी': 3,
         'वर्मा\nसिराजुद्दौला\nसुभाष': 3,
         'बोस\nसुब्रमण्य': 3,
         'भारती\nसूर्य': 3,
         'सेन\nश्यामाप्रसाद': 3,
         'मुखर्जी\nतारकनाथ': 3,
         'दास\nतात्या': 3,
         'टोपे\nतिरुपुर': 3,
         'कुमारन\nउबैदुल्लाह': 3,
         'सिंधी\nवी०': 3,
         'ओ०': 3,
         'चिदम्बरम': 3,
         'पिल्लै\nवी': 3,
         'मेनन\nवल्लभभाई': 3,
         'पटेल\nवंचिनाथन\nविनायक': 3,
         'सावरकर\nवीरेन्द्रनाथ': 3,
         'चट्टोपाध्याय\nयशवंतराव': 3,
         'होलकर\nऔर': 3,
         'देखे\nब्रिटिश': 3,
         'नेत

In [39]:
import os
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer

# Define the directory path
dir_path = '/content/pre_processed_corpus'

documents = []

for filename in os.listdir(dir_path):
    if filename.startswith('corpus_') and filename.endswith('.txt'):
        file_path = os.path.join(dir_path, filename)
        with open(file_path, 'r', encoding='utf-8') as f:
            document = f.read()
            documents.append(document)

vectorizer = CountVectorizer(stop_words='english')

X = vectorizer.fit_transform(documents)

feature_names = vectorizer.get_feature_names_out()

df = pd.DataFrame(X.toarray(), columns=feature_names)

df.to_csv('/content/output/Bag_of_words_scikit.csv', index=True, encoding='utf-8')

print(df)

# Save the feature names to vocabulary.txt
with open('/content/output/vocabulary_scikit.txt', 'w', encoding='utf-8') as f:
    for feature in feature_names:
        f.write(feature + '\n')

   अक  अकबर  अख  अखण  अखब  अग  अगण  अगम  अगय  अगर  ...  ८१७२२९१३८८  ८२  ८४०००  \
0  58     3   6    0    2   7    1    0    0   13  ...           1   1      0   
1   3     0   0    0    1   1    0    0    0   15  ...           0   0      0   
2  21     0   1    0    6   6    0    0    1   22  ...           0   0      0   
3  30     0  15    2    1   5    0    2    0    0  ...           0   0      1   

   ८६  ८८१८२०  ८८३९३८  ८९  ९व  ९१७१९९३०२६००एच  ९२  
0   0       0       1   1   0               1   1  
1   0       0       0   0   0               0   0  
2   1       0       0   0   0               0   0  
3   0       1       0   0   2               0   0  

[4 rows x 2665 columns]
